In [ ]:
#!/usr/bin/env python
# coding: utf-8
# In[1]:
import cv2
from PIL import Image
import numpy as np
import csv
# In[2]:
def ScoreImage(imgGray):
    score=[]
    imgGray = imgGray.astype("uint8")
    thresh_img = cv2.threshold(imgGray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]
    numOfWhite=np.count_nonzero(thresh_img == 255)
    score.append(numOfWhite/(thresh_img.shape[0]*thresh_img.shape[1]))
    return np.max(score)
# In[3]
def detectNoise(imgGray):
  imgGray = imgGray.astype("uint8")
  thresh_img = cv2.threshold(imgGray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]
  #cv2.imshow('Binary',thresh_img)
  def iterateOntheImage(start1,start2,thresh_img,image,step1,step2):
      windowSize1=int(thresh_img.shape[0]/step1)
      windowSize2= int(thresh_img.shape[1]/step2)
      pixel_value=thresh_img[start1:windowSize1+start1,0+start2:windowSize2+start2]
      numOfWhite=np.count_nonzero(pixel_value == 255)
      score=numOfWhite/(windowSize1*windowSize2)
      return score
  start_r=0
  start_c=0
  score=[]
  windowSize1=int(thresh_img.shape[0]/2)
  windowSize2= int(thresh_img.shape[1]/2)
  Maxscore=0
  total=0
  while(start_c+windowSize2<thresh_img.shape[1]):
      start_r=0
      while(start_r+windowSize1<thresh_img.shape[0]):
          score.append(iterateOntheImage(start_r,start_c,thresh_img,image,2,2))
          total+=1 
          start_r=start_r+windowSize1
      start_c=start_c+windowSize2
  Maxscore=np.max(score)
 # print(Maxscore)
  if(Maxscore>0.3):
    return 1
  else:
    return 0
# In[4]:
vidcap=[]
vidcap.append ( cv2.VideoCapture('2.mp4'))
vidcap.append ( cv2.VideoCapture('3.mp4'))
vidcap.append ( cv2.VideoCapture('4.mp4'))
vidcap.append ( cv2.VideoCapture('10.mp4'))
vidcap.append ( cv2.VideoCapture('11.mp4'))
#vidcap.append ( cv2.VideoCapture('12.mp4'))
vidcap.append ( cv2.VideoCapture('13.mp4'))
# In[5]
filename = "data.csv"
filename2 = "label.csv"
fields = ['max', 'std', 'score of binery', 'average'] 
fields2 = ['result'] 
allfeacher=[]
label=[]
grey_levels = 256
windowsize_r = 96
windowsize_c = 96
count=0
for j in range(len(vidcap)):
  movie=vidcap[j]
  print(movie)
  success,image = movie.read()
  count_frame=0
  while success:
    cv2.imwrite("frame%d.jpg" %count_frame, image)     # save frame as JPEG file      
    success,image = movie.read()
    count_frame+=1
    
 
  NumOfFrame=count_frame


  # Crop out the window and calculate the histogram
  for frame in range(NumOfFrame):
    image = cv2.imread("frame"+str(frame)+".jpg",cv2.IMREAD_COLOR)
    #print(frame)
    test_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    for r in range(0,test_image.shape[0] - windowsize_r, windowsize_r):
      for c in range(0,test_image.shape[1] - windowsize_c, windowsize_c):
          window = test_image[r:r+windowsize_r,c:c+windowsize_c]
          im = Image.fromarray(window)
          im.save("block"+" "+str(count)+".jpg")  
          R, G, B = window[:,:,0], window[:,:,1], window[:,:,2]
          imgGray = 0.2989 * R + 0.5870 * G + 0.1140 * B
          #print(count)
          feacher=[]
          l=[]
          feacher.append(np.max(imgGray))
          feacher.append(np.std(imgGray))
          feacher.append(ScoreImage(imgGray))
          feacher.append(np.average(imgGray))
          allfeacher.append(feacher)
       
          if(detectNoise(imgGray)==1):
            l.append("1")
            label.append(l)
          else:
            l.append("0")
            label.append(l)
          count += 1

with open(filename, 'w', encoding='UTF8', newline='') as csvfile: 
    # creating a csv writer object 
    csvwriter = csv.writer(csvfile) 
        
    # writing the fields 
    csvwriter.writerow(fields) 
        
    # writing the data rows 
    csvwriter.writerows(allfeacher)
with open(filename2, 'w', encoding='UTF8', newline='') as csvfile: 
    # creating a csv writer object 
    csvwriter = csv.writer(csvfile) 
        
    # writing the fields 
    csvwriter.writerow(fields2) 
        
    # writing the data rows 
    csvwriter.writerows(label)

<VideoCapture 000001D5ADE0C610>
<VideoCapture 000001D5AC408CB0>
<VideoCapture 000001D5AC408D30>
<VideoCapture 000001D5AC408C30>
<VideoCapture 000001D5AC408C50>
<VideoCapture 000001D5AC408D70>
